In [1]:
from models import (
    train,
    iterated_model_unrolled, 
    iterated_model_while_op, 
    cnn_model
)
import tensorflow as tf
import numpy as np
from functools import partial
from utils import Timer

tf.logging.set_verbosity(tf.logging.ERROR)

%rm -r ./data/

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

steps = 1e2

def train_on_mnist(*args, **kwargs):
    with Timer() as timer:
        results = train(eval_data, eval_labels, train_data, train_labels, steps, *args, **kwargs)
    return (timer.elapsed_seconds, *results)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [3]:
cnn_model_results = train_on_mnist(cnn_model)
cnn_model_results

(5.181734431000223,
 {'accuracy': 0.117, 'loss': 2.2781184, 'global_step': 100})

In [4]:
iterated_model_unrolled_results = train_on_mnist(iterated_model_unrolled, params={'iterations': 5, 'channels': 5})
iterated_model_unrolled_results

(5.272688242001095,
 {'accuracy': 0.4271, 'loss': 2.0145547, 'global_step': 100})

In [5]:
iterated_model_while_op_results = train_on_mnist(iterated_model_while_op, params={'iterations': 5, 'channels': 5})
iterated_model_while_op_results

(7.878105690000666,
 {'accuracy': 0.4536, 'loss': 2.028667, 'global_step': 100})

In [6]:
def iteration_study(iterations):
    return train_on_mnist(
        iterated_model_unrolled,
        f'iterated_model_unrolled_iterations_{iterations}',
        {
            'iterations': iterations,
            'channels': 5
        }
    )

In [7]:
iterations = [
    0,  # Trivial case
    1,
    2,
    5,
    10,
    20
]
results = list(map(iteration_study, iterations))
results

[(1.9715592189986637,
  {'accuracy': 0.1753, 'loss': 2.2542458, 'global_step': 100}),
 (2.6126796480002668,
  {'accuracy': 0.3519, 'loss': 2.159173, 'global_step': 100}),
 (3.2279673499997443,
  {'accuracy': 0.4362, 'loss': 2.0282006, 'global_step': 100}),
 (5.237754132000191,
  {'accuracy': 0.35, 'loss': 2.15383, 'global_step': 100}),
 (8.586066130999825,
  {'accuracy': 0.1913, 'loss': 2.2482212, 'global_step': 100}),
 (15.12923201600097,
  {'accuracy': 0.1135, 'loss': 2.3025482, 'global_step': 100})]

In [8]:
def channel_study(channels):
    return train_on_mnist(
        iterated_model_unrolled,
        f'iterated_model_unrolled_channels_{channels}',
        {
            'iterations': 5,
            'channels': channels
        }
    )

In [9]:
channels = [
    1,
    2,
    5,
    10,
    20
]
results = list(map(channel_study, channels))
results

[(3.059595332000754,
  {'accuracy': 0.1135, 'loss': 2.3025558, 'global_step': 100}),
 (3.5366202920013166,
  {'accuracy': 0.6413, 'loss': 1.3299961, 'global_step': 100}),
 (5.249716188000093,
  {'accuracy': 0.3882, 'loss': 2.1251016, 'global_step': 100}),
 (7.751644087999011,
  {'accuracy': 0.4337, 'loss': 2.0537674, 'global_step': 100}),
 (12.64566842899876,
  {'accuracy': 0.3524, 'loss': 2.2000964, 'global_step': 100})]